In [1]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!source /content/drive/MyDrive/colab_env/bin/activate

In [ ]:
! pip install geemap

In [44]:
import xarray as xr
import ee
import itertools
import pickle
import pandas as pd

In [45]:
LAT_MIN = 25
LAT_MAX = 45
LON_MIN = -125
LON_MAX = -65

In [46]:
soil_wc = xr.open_dataset('/content/drive/MyDrive/w/soil_water_content_1979001.nc')


In [47]:
service_account = 'test-175@ee-mohammadnejadmehdi77.iam.gserviceaccount.com'

credentials = ee.ServiceAccountCredentials(
    email=service_account,
    key_file="/content/drive/MyDrive/w/private-key.json"
)

ee.Initialize(credentials)


In [48]:
# Soil depths [in cm] where we have data.
olm_depths = [0, 10, 30, 60, 100, 200]

# Names of bands associated with reference depths.
olm_bands = ["b" + str(sd) for sd in olm_depths]

def get_soil_prop(param):
    """
    This function returns soil properties image
    param (str): must be one of:
        "sand"     - Sand fraction
        "clay"     - Clay fraction
        "orgc"     - Organic Carbon fraction
    """
    if param == "sand":  # Sand fraction [%w]
        snippet = "OpenLandMap/SOL/SOL_SAND-WFRACTION_USDA-3A1A1A_M/v02"
        # Define the scale factor in accordance with the dataset description.
        scale_factor = 1 * 0.01

    elif param == "clay":  # Clay fraction [%w]
        snippet = "OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02"
        # Define the scale factor in accordance with the dataset description.
        scale_factor = 1 * 0.01

    elif param == "orgc":  # Organic Carbon fraction [g/kg]
        snippet = "OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02"
        # Define the scale factor in accordance with the dataset description.
        scale_factor = 5 * 0.001  # to get kg/kg
    else:
        return print("error")

    # Apply the scale factor to the ee.Image.
    dataset = ee.Image(snippet).multiply(scale_factor)

    return dataset


# Image associated with the sand content.
sand = get_soil_prop("sand")

# Image associated with the clay content.
clay = get_soil_prop("clay")

# Image associated with the organic carbon content.
orgc = get_soil_prop("orgc")

In [31]:
import geemap
import numpy as np
import matplotlib.pyplot as plt

In [43]:
import wxee
crs = 4326
# LAT_MIN = 43.30
# LAT_MAX = 44.60
# LON_MIN = -109.40
# LON_MAX = -107.40

# shape = ee.Geometry.Rectangle([LON_MAX,LAT_MIN,LON_MIN,LAT_MAX])



shape1 = ee.Geometry.Rectangle([LON_MAX+10,LAT_MIN,LON_MIN,LAT_MAX-10])






s = sand.clip(shape)

s_s = s.set("system:time_start", ee.Date("1979-01-01"))

# sss = s_s.wx.to_xarray(scale=250, crs=f'EPSG:{crs}')

qgis_rect = QgsRectangle(shape.getInfo()['coordinates'][0])

# rgb_img = geemap.ee_to_numpy(s_s, region=shape)


# s_s.wx.to_xarray(scale=250)



NameError: ignored

In [ ]:
lat_of_points = list(soil_wc.y.values.flatten())
lon_of_points = list(soil_wc.x.values.flatten())

grid = [(float(i), float(j)) for i, j in itertools.product(lon_of_points, lat_of_points)]
x_values = [t[0] for t in grid]
y_values = [t[1] for t in grid]

In [ ]:
none_profile = {
    'b0': pd.NA,
    'b10': pd.NA,
    'b100': pd.NA,
    'b200': pd.NA,
    'b30': pd.NA,
    'b60': pd.NA
}
# scale of original data such as tmin and precipitation - gloH2o
scale = 12128.507

def local_profile(dataset, points, buffer):
  """Gets the properties of a list of points.

  Args:
    dataset: The Earth Engine dataset to query.
    points: A list of points.
    buffer: The buffer size to use when sampling the dataset.

  Returns:
    A list of dictionaries, where each dictionary contains the properties of a point.
  """

  profiles = []
  for point in points:
    try :
      poi = ee.Geometry.Point(point[0], point[1])
      prop = dataset.sample(poi, buffer).select(olm_bands).getInfo()
      profile = prop["features"][0]["properties"]
    except IndexError:
      profile = none_profile

    profiles.append(profile)

  return profiles

orgm = orgc.multiply(1.724)
point = list(zip(x_values, y_values))

list_org = local_profile(
    dataset = orgm,
    points = point,
    buffer = scale
)
list_sand = local_profile(
    dataset = sand,
    points = point,
    buffer = scale
)
list_clay = local_profile(
    dataset = clay,
    points = point,
    buffer = scale
)

In [ ]:
with open('/content/drive/MyDrive/WaterBalance_new/notebook/ETa_pyet_hargreaves_mahdoode_mashhad_4326.pkl', 'rb') as file:
    sample_data = pickle.load(file)

In [ ]:
sample_data

<xarray.Dataset>
Dimensions:          (time: 1, y: 13, x: 18)
Coordinates:
  * x                (x) float32 58.35 58.45 58.55 58.65 ... 59.85 59.95 60.05
  * y                (y) float32 37.05 36.95 36.85 36.75 ... 36.05 35.95 35.85
  * time             (time) datetime64[ns] 2022-01-31
    spatial_ref      int64 0
Data variables:
    air_temperature  (time, y, x) float32 nan nan -6.875 -6.188 ... nan nan nan
    et_covered       (time, y, x) float64 nan nan 0.0 0.0 ... nan nan nan nan
    e_noncovered     (time, y, x) float64 nan nan 0.3705 0.3337 ... nan nan nan
    et_total         (time, y, x) float64 nan nan 0.3705 0.3337 ... nan nan nan
    shapefile        (y, x) float64 nan nan 0.0 0.0 0.0 ... nan nan nan nan nan
Attributes:
    history:          Created on 2022-02-06 07:16
    input_data_hash:  a1123a6e0d2788e1883ec8f01cc52110d92449cef4cf1591984fe91...

In [ ]:
lat_of_points = list(sample_data.y.values.flatten())
lon_of_points = list(sample_data.x.values.flatten())

grid = [(float(i), float(j)) for i, j in itertools.product(lon_of_points, lat_of_points)]
x_values = [t[0] for t in grid]
y_values = [t[1] for t in grid]


In [ ]:
coordinates = [ee.Feature(ee.Geometry.Point([ee.Number(lon), ee.Number(lat)])) for lat, lon in zip(y_values, x_values)]
samples = sand.sampleRegions(
  collection=coordinates,
  scale=12128.507,
  geometries=True # You said you wait to show the points, so you might want to include the geometries?
)



features = samples.toList(samples.size())


ip = features.size().getInfo()
ip

EEException: ignored

In [ ]:
LAT_MIN = 25
LAT_MAX = 45
LON_MIN = -125
LON_MAX = -65

In [ ]:
none_profile = {
    'b0': pd.NA,
    'b10': pd.NA,
    'b100': pd.NA,
    'b200': pd.NA,
    'b30': pd.NA,
    'b60': pd.NA
}

In [ ]:


x = [-124.94999694824219, -124.84999084472656]
y = [25.049999237060547, 25.14999771118164]
point = [ee.Feature(ee.Geometry.Point([ee.Number(lon), ee.Number(lat)])) for lat, lon in zip(y, x)]

samples = sand.sampleRegions(
  collection=point,
  scale=12128.507,
  geometries=True # You said you wait to show the points, so you might want to include the geometries?
)


li = []

features = samples.toList(samples.size())


for i in range(features.size().getInfo()):
  try:
    l = ee.Feature(features.get(i)).getInfo()["properties"]

  except ee.EEException as e:
    l = none_profile  # Assign an empty list or any other default value
    li.append(l)







features.size()


# features = samples.toList(samples.size())
# li = [ee.Feature(features.get(i)).getInfo()["properties"] for i in range(features.size().getInfo())]


# li = list(map(lambda i: ee.Feature(features.get(i)).getInfo()["properties"], range(features.size().getInfo())))


TypeError: ignored

In [ ]:
import wxee

coordinates = [ee.Feature(ee.Geometry.Point([ee.Number(lon), ee.Number(lat)])) for lat, lon in zip(y_values, x_values)]
samples = sand.sampleRegions(
  collection=coordinates,
  scale=12128.507,
  geometries=True
)


features = samples.toList(samples.size())

# li = [ee.Feature(features.get(i)).getInfo()["properties"] for i in range(features.size().getInfo())]

# li = list(map(lambda i: ee.Feature(features.get(i)).getInfo()["properties"], range(features.size().getInfo())))


li = []
for i in range(234):
  element = features.get(i)
  profile = element.getInfo()
  l = profile["properties"]
  li.append(l)
#   # break

li

[{'b0': 0.34,
  'b10': 0.33,
  'b100': 0.35000000000000003,
  'b200': 0.37,
  'b30': 0.33,
  'b60': 0.33},
 {'b0': 0.41000000000000003,
  'b10': 0.4,
  'b100': 0.44,
  'b200': 0.47000000000000003,
  'b30': 0.4,
  'b60': 0.42},
 {'b0': 0.42,
  'b10': 0.41000000000000003,
  'b100': 0.42,
  'b200': 0.44,
  'b30': 0.4,
  'b60': 0.41000000000000003},
 {'b0': 0.46,
  'b10': 0.45,
  'b100': 0.48,
  'b200': 0.49,
  'b30': 0.45,
  'b60': 0.46},
 {'b0': 0.42,
  'b10': 0.42,
  'b100': 0.44,
  'b200': 0.46,
  'b30': 0.41000000000000003,
  'b60': 0.41000000000000003},
 {'b0': 0.52,
  'b10': 0.51,
  'b100': 0.54,
  'b200': 0.54,
  'b30': 0.51,
  'b60': 0.52},
 {'b0': 0.48,
  'b10': 0.47000000000000003,
  'b100': 0.5,
  'b200': 0.51,
  'b30': 0.46,
  'b60': 0.47000000000000003},
 {'b0': 0.56,
  'b10': 0.55,
  'b100': 0.59,
  'b200': 0.6,
  'b30': 0.55,
  'b60': 0.55},
 {'b0': 0.5700000000000001,
  'b10': 0.55,
  'b100': 0.6,
  'b200': 0.6,
  'b30': 0.56,
  'b60': 0.5700000000000001},
 {'b0': 0.6, 'b1

In [ ]:
import numpy as np

number_of_x = len(list(sample_data.x.values.flatten()))
number_of_y = len(list(sample_data.y.values.flatten()))

sand_list_10cm = list(map(lambda dict: dict['b10'], li))
sand_list_10cm_array = np.array(sand_list_10cm)
sand_list_10cm_array = sand_list_10cm_array.reshape(1,number_of_y,number_of_x)
ds_array_15 = sample_data.assign(sand_10cm=(['time', 'y', 'x'], sand_list_10cm_array))
ds_array_15

<xarray.Dataset>
Dimensions:          (time: 1, y: 13, x: 18)
Coordinates:
  * x                (x) float32 58.35 58.45 58.55 58.65 ... 59.85 59.95 60.05
  * y                (y) float32 37.05 36.95 36.85 36.75 ... 36.05 35.95 35.85
  * time             (time) datetime64[ns] 2022-01-31
    spatial_ref      int64 0
Data variables:
    air_temperature  (time, y, x) float32 nan nan -6.875 -6.188 ... nan nan nan
    et_covered       (time, y, x) float64 nan nan 0.0 0.0 ... nan nan nan nan
    e_noncovered     (time, y, x) float64 nan nan 0.3705 0.3337 ... nan nan nan
    et_total         (time, y, x) float64 nan nan 0.3705 0.3337 ... nan nan nan
    shapefile        (y, x) float64 nan nan 0.0 0.0 0.0 ... nan nan nan nan nan
    sand_10cm        (time, y, x) float64 0.33 0.4 0.41 0.45 ... 0.4 0.4 0.46
Attributes:
    history:          Created on 2022-02-06 07:16
    input_data_hash:  a1123a6e0d2788e1883ec8f01cc52110d92449cef4cf1591984fe91...

In [ ]:
element = features.get(233)
l = element.getInfo()
l["properties"]


{'b0': 0.46, 'b10': 0.46, 'b100': 0.48, 'b200': 0.49, 'b30': 0.46, 'b60': 0.46}

In [ ]:
samples["features"][0]["properties"]

{'b0': 0.34,
 'b10': 0.33,
 'b100': 0.35000000000000003,
 'b200': 0.37,
 'b30': 0.33,
 'b60': 0.33}

In [ ]:
points = list(zip(x_values, y_values))

profiles = []

for point in points:
    poi = ee.Geometry.Point(point[0], point[1])
    prop = sand.sample(poi, 12128.507).getInfo()
    # prop = sand.sample(poi, 12128.507).select(olm_bands).getInfo()
    profile = prop["features"][0]["properties"]

    # profile = prop["features"][0]["properties"]
    # # profile_1 = {key: round(val, 3) for key, val in profile.items()}
    profiles.append(profile)


In [ ]:

roi = ee.Geometry.Rectangle([LON_MAX,LAT_MIN,LON_MIN,LAT_MAX])
point_count = 100  # Number of points in the grid
grid_points = ee.FeatureCollection.randomPoints(region=roi, points=point_count)
samples = sand.sampleRegions(collection=grid_points, scale=12128.507)
prop = samples.getInfo()
profile = prop["features"][0]["properties"]
profile

{'b0': 0.58, 'b10': 0.58, 'b100': 0.63, 'b200': 0.64, 'b30': 0.59, 'b60': 0.61}

In [ ]:
import geemap

Map = geemap.Map()
Map.add_basemap('HYBRID')


coordinates = [ee.Feature(ee.Geometry.Point([ee.Number(lon), ee.Number(lat)])) for lat, lon in zip(y_values, x_values)]

samples = sand.sampleRegions(
  collection=coordinates,
  scale=12128.507,
  geometries=False # You said you wait to show the points, so you might want to include the geometries?
)


# Map.centerObject(samples, 9)
Map.addLayer(samples, {}, 'samples')
Map

EEException: ignored

In [ ]:
import geopandas as gpd
coordinates = [ee.Feature(ee.Geometry.Point([ee.Number(lon), ee.Number(lat)])) for lat, lon in zip(y_values, x_values)]

samples = sand.sampleRegions(
  collection=coordinates,
  scale=12128.507,
  geometries=False # You said you wait to show the points, so you might want to include the geometries?
)
# gdf = geemap.ee_to_geopandas(samples)
# dataset = xr.Dataset.from_dataframe(gdf)




# Set the batch size
batch_size = 10000000

# Initialize an empty GeoDataFrame
gdf = gpd.GeoDataFrame()

# Iterate through the FeatureCollection in batches
for i in range(0, samples.size().getInfo(), batch_size):
    start = i
    end = i + batch_size

    # Get a batch of features
    batch = samples.toList(batch_size, start)

    # Convert the batch to a GeoDataFrame
    batch_gdf = geemap.ee_to_geopandas(batch)

    # Append the batch GeoDataFrame to the main GeoDataFrame
    gdf = gdf.append(batch_gdf, ignore_index=True)

# Convert the GeoDataFrame to an xarray dataset
dataset = xr.Dataset.from_dataframe(gdf)

# Print the xarray dataset
print(dataset)

EEException: ignored

In [ ]:
points = ee.FeatureCollection(ee.List.sequence(1, 200)
  .map(
    lambda i: ee.Feature(ee.Geometry.Point([ee.Number(i), 0]))
  )
)

image = ee.Image([
  ee.Image.random(1),
  ee.Image.random(2),
  ee.Image.random(3)
]).reduce(ee.Reducer.sum()) # Simple way to sum all bands

samples = image.sampleRegions(
  collection=points,
  scale=30,
  geometries=True # You said you wait to show the points, so you might want to include the geometries?
)

gdf = geemap.ee_to_geopandas(samples)
dataset = xr.Dataset.from_dataframe(gdf)
dataset

ee.featurecollection.FeatureCollection

<xarray.Dataset>
Dimensions:   (index: 200)
Coordinates:
  * index     (index) int64 0 1 2 3 4 5 6 7 ... 192 193 194 195 196 197 198 199
Data variables:
    geometry  (index) object POINT (0.9999596585176455 0.0001347472926179) .....
    sum       (index) float64 2.387 1.732 0.5886 1.913 ... 1.685 1.459 1.273

In [ ]:
coordinates = [ee.Feature(ee.Geometry.Point([ee.Number(lon), ee.Number(lat)])) for lat, lon in zip(y_values, x_values)]

samples = sand.sampleRegions(
  collection=coordinates,
  scale=12128.507,
  geometries=True # You said you wait to show the points, so you might want to include the geometries?
)

In [ ]:
samples